# IN PROGRESS ...

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [14]:
url = "https://www.foxsports.com/soccer/mexico-men-team-roster"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [36]:
rows = soup.find_all('tr')
rows[1].find('td')

<td class="cell-entity fs-18 lh-1pt67" data-index="0"><div class="table-entity"><!-- --><!-- --><a class="table-entity-name ff-s" href="/soccer/carlos-acevedo-player"><h3>Carlos Acevedo</h3></a><sup class="table-superscript mg-b-5 ffn-11 ls-pt5">#12</sup></div></td>

In [38]:
rows[1].find('h3').get_text()

'Carlos Acevedo'

In [40]:
soup.find_all('tr')[1]

<tr><td class="cell-entity fs-18 lh-1pt67" data-index="0"><div class="table-entity"><!-- --><!-- --><a class="table-entity-name ff-s" href="/soccer/carlos-acevedo-player"><h3>Carlos Acevedo</h3></a><sup class="table-superscript mg-b-5 ffn-11 ls-pt5">#12</sup></div></td><td class="cell-text ff-h" data-index="1">
    G
  </td><td class="cell-text ff-h" data-index="2">
    26
  </td><td class="cell-text ff-h" data-index="3">
    5'11"
  </td><td class="cell-text ff-h" data-index="4">
    149 lbs
  </td></tr>